In [ ]:
Build a one node model with 3 Generators and 2 storages

In [ ]:
import pypsa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('bmh')
%matplotlib inline

In [ ]:
def build_network():
    network = pypsa.Network()
    snapshots = pd.date_range("2014-01-01", "2014-12-31 23:00", freq="H")

    network.set_snapshots(snapshots)

    network.snapshot_weightings = pd.Series(float("1"), index=network.snapshots)
    GNwt = pd.read_csv("data/ninja_europe_wind_v1/ninja_wind_europe_v1.1_future_nearterm_on-offshore.csv", index_col=0)
    GNwt = GNwt.loc["2014-01-01":"2015-01-01", "DE_ON"]
    snapshots = pd.date_range("2014-01-01", "2014-12-31 23:00", freq="H")
    GNwt = GNwt.reindex(snapshots, method="nearest")
    GSst = pd.read_csv("data/ninja_europe_pv_v1/ninja_pv_europe_v1.1_merra2.csv", index_col=0)

    # GSst = GSst.loc["2014-01-01":"2015-01-01",:]*100
    GSst = GSst.loc["2014-01-01":"2015-01-01", "DE"]
    GSst = GSst.reindex(snapshots, method="nearest")
    
    # # # # bug-fix until PyPSA 0.16.2
    network.snapshot_weightings.name = "weightings"

    network.add("Bus", "DE")
    network.add("Load", "load",
                bus="DE",
                p_set=9000)

    network.add("Generator", "solar",
                bus="DE",
                p_max_pu=GSst,
                p_nom_extendable=True,
                marginal_cost=0.1,
                # Small cost to prefer curtailment to destroying energy in storage, solar curtails before wind
                capital_cost=35817)

    network.add("Generator", "wind",
                bus="DE",
                p_max_pu=GNwt,
                p_nom_extendable=True,
                marginal_cost=0.2,
                # Small cost to prefer curtailment to destroying energy in storage, solar curtails before wind
                capital_cost=90596)

    network.add("Generator", "OCGT",
                bus="DE",
                p_nom=1000,
                p_nom_extendable=False,
                marginal_cost=580,
                efficiency=0.39,
                p_max_pu=1)

    network.add("Bus", "DE battery")

    network.add("Store", "DE battery storage",
                bus="DE battery",
                e_nom_extendable=True,
                e_max_pu=1,
                e_cyclic=True,
                capital_cost=10534)

    network.add("Link", "DE battery charge",
                bus0="DE",
                bus1="DE battery",
                efficiency=0.9,
                # p_nom_max=100,
                p_nom_extendable=True,
                capital_cost=14045)

    network.add("Link", "DE battery discharge",
                bus0="DE battery",
                bus1="DE",
                p_nom_extendable=True,
                efficiency=0.9)

    network.add("Bus",
                "DE H2",
                carrier="H2")

#     network.add("Load", "hydrogen_load",
#                 bus="hydrogen",
#                 p_set=200)

    network.add("Link",
                "DE H2 electrolysis",
                bus1="DE H2",
                bus0="DE",
                # p_nom=1000,
                p_nom_extendable=True,
                marginal_cost=0.7,
                efficiency=0.8,
                capital_cost=47073)

    network.add("Link",
                "DE H2 to power",
                bus0="DE H2",
                bus1="DE",
                p_nom_extendable=True,
                efficiency=0.6,
                capital_cost=40452)  # NB: fixed cost is per MWel

    network.add("Store",
                "DE H2 storage",
                bus="DE H2",
                e_nom_extendable=True,
                e_cyclic=True,
                capital_cost= 855)
    return network

def set_parameters_from_optimized(n,n_optim):
    links_i = n.links.index[n.links.p_nom_extendable]
    n.links.loc[links_i, 'p_nom'] = n_optim.links['p_nom_opt'].reindex(links_i, fill_value=0.)
    n.links.loc[links_i, 'p_nom_extendable'] = False

    gen_extend_i = n.generators.index[n.generators.p_nom_extendable]
    n.generators.loc[gen_extend_i, 'p_nom'] = n_optim.generators['p_nom_opt'].reindex(gen_extend_i, fill_value=0.)
    n.generators.loc[gen_extend_i, 'p_nom_extendable'] = False


    stores_extend_i = n.stores.index[n.stores.e_nom_extendable]
    n.stores.loc[stores_extend_i, 'e_nom'] = n_optim.stores['e_nom_opt'].reindex(stores_extend_i,fill_value=0.)
    n.stores.loc[stores_extend_i, 'e_nom_extendable'] = False
    
    return n

def prepare_network(n):
    n.add("Generator", "load_shedding",
                bus="DE",
                carrier='load',
                marginal_cost=1e6,  # Eur/kWh
                p_nom=1e9  # kW
                )
   # n.loads["p_set"]=n.loads["p_set"]+50
    return n

def get_storeage_bid_de(n, snapshots, bus, charger, discharger):
    charge_efficiency = n.links.efficiency[charger]
    discharge_efficiency = n.links.efficiency[discharger]
    #λ = n.stores_t.mu_state_of_charge.loc[snapshots,"DE H2 storage"]
    λ = n.buses_t.marginal_price.loc[snapshots, bus] 
    if "mu_lower_e" in n.stores_t:
        if bus + " storage" in n.stores_t.mu_lower_e.columns:
            mu_lower = n.stores_t.mu_lower_e.loc[snapshots, bus + " storage"]
        else: mu_lower =0
        if bus + " storage" in n.stores_t.mu_upper_e.columns:
            mu_upper = n.stores_t.mu_upper_e.loc[snapshots, bus + " storage"]
        else: mu_upper=0
    charging_bid = charge_efficiency*λ
    discharging_bid =  λ#/discharge_efficiency
    return charging_bid, discharging_bid  



def solve_network_de_rh_sp(n, opti_n, add_h2_bidding, add_battery_didding, rh_each_snapshot, rh_2w):
    solver_name = "gurobi"

    freq = 1


    length = len(n.snapshots)
    if rh_each_snapshot == True:
        loop_num = len(n.snapshots)-1
        kept = 1
        overlap = 0
        
    if rh_2w == True:
        window = 14 * 24 // freq
        overlap = 7 * 24 // freq
        kept = window - overlap
        loop_num = length // kept
        
        
    if add_h2_bidding == True:
        h2_charging_bid, h2_discharging_bid = get_storeage_bid_de(opti_n,n.snapshots, "DE H2", "DE H2 electrolysis", "DE H2 to power")
        n.links_t.marginal_cost["DE H2 electrolysis"] = 0
        n.links_t.marginal_cost.loc[n.snapshots, "DE H2 electrolysis"] = - h2_charging_bid
        n.links_t.marginal_cost["DE H2 to power"] = 0
        n.links_t.marginal_cost.loc[n.snapshots, "DE H2 to power"] = h2_discharging_bid
    if add_battery_didding == True:
        battery_charging_bid, battery_discharging_bid = get_storeage_bid_de(opti_n,n.snapshots, "DE battery", "DE battery charge", "DE battery discharge")
        n.links_t.marginal_cost["DE battery charge"] = 0
        n.links_t.marginal_cost.loc[n.snapshots, "DE battery charge"] = - battery_charging_bid
        n.links_t.marginal_cost["DE battery discharge"] = 0
        n.links_t.marginal_cost.loc[n.snapshots, "DE battery discharge"] = battery_discharging_bid

    for i in range(loop_num):
        # set initial state of charge
        snapshots = n.snapshots[i * kept:(i + 1) * kept + overlap]        
        if i == 0:
            n.stores.e_initial = opti_n.stores_t.e.iloc[0]
            n.storage_units.state_of_charge_initial = opti_n.storage_units_t.state_of_charge.iloc[-1]
        else:
            n.stores.e_initial = n.stores_t.e.iloc[i * kept - 1]

            n.storage_units.state_of_charge_initial = n.storage_units_t.state_of_charge.iloc[i * kept - 1]
        #n.lopf(pyomo=False, solver_name="gurobi", solver_logfile=None, snapshots=snapshots,  keep_shadowprices=True)
        pypsa.linopf.network_lopf(n,snapshots,solver_name="gurobi", solver_logfile=None,
                                     keep_shadowprices=True)
    
    return n

In [ ]:
# Perfect foresight:  sovled network
#n_de = pypsa.Network("data/de.nc")
n_de = build_network()

In [ ]:
n_network = n_de.copy()
n_network.lopf(pyomo=False, solver_name="gurobi", solver_logfile=None, keep_shadowprices=True )

In [ ]:
n_operation = n_de.copy()
n_operation = set_parameters_from_optimized(n_operation,n_network)
n_operation = prepare_network(n_operation)
#operation_network.loads["p_set"]=operation_network.loads["p_set"]+50
n_operation.lopf(pyomo=False, solver_name="gurobi", solver_logfile=None, keep_shadowprices=True)

In [ ]:
n_de_rh = n_de.copy()
#n_de_rh = drop_solved_components(n_de_rh)
n_de_rh = set_parameters_from_optimized(n_de_rh, n_operation)
n_de_rh = prepare_network(n_de_rh)
n_de_rh.stores.e_cyclic = False
n_de_rh = solve_network_de_rh_sp(n_de_rh, n_operation, add_h2_bidding = False, add_battery_didding = False, rh_each_snapshot = False, rh_2w = True)

In [ ]:
# n_de_rh_sp_h2 = n_de.copy()
# #n_de_rh_sp_h2 = drop_solved_components(n_de_rh_sp_h2)
# n_de_rh_sp_h2 = set_parameters_from_optimized(n_de_rh_sp_h2, n_operation)
# n_de_rh_sp_h2 = prepare_network(n_de_rh_sp_h2)
# n_de_rh_sp_h2.stores.e_cyclic = False
# n_de_rh_sp_h2 = solve_network_de_rh_sp(n_de_rh_sp_h2, n_operation, add_h2_bidding = True, add_battery_didding = False, rh_each_snapshot = True, rh_2w = False)

In [ ]:
n_de_rh_2w_h2 = n_de.copy()
#n_de_rh_2w_h2 = drop_solved_components(n_de_rh_2w_h2)
n_de_rh_2w_h2 = set_parameters_from_optimized(n_de_rh_2w_h2, n_operation)
n_de_rh_2w_h2 = prepare_network(n_de_rh_2w_h2)
n_de_rh_2w_h2.stores.e_cyclic = False
n_de_rh_2w_h2 = solve_network_de_rh_sp(n_de_rh_2w_h2, n_operation, add_h2_bidding = True, add_battery_didding = False, rh_each_snapshot = False, rh_2w = True)

In [ ]:
# n_de_rh_sp_all = n_de.copy()
# n_de_rh_sp_all = set_parameters_from_optimized(n_de_rh_sp_all, n_operation)
# n_de_rh_sp_all = prepare_network(n_de_rh_sp_all)
# n_de_rh_sp_all.stores.e_cyclic = False
# n_de_rh_sp_all = solve_network_de_rh_sp(n_de_rh_sp_all, n_operation, add_h2_bidding = True, add_battery_didding = True, rh_each_snapshot = True, rh_2w = False)

In [ ]:
n_de_rh_2w_all = n_de.copy()
#n_de_rh_2w_all = drop_solved_components(n_de_rh_2w_all)
n_de_rh_2w_all = set_parameters_from_optimized(n_de_rh_2w_all, n_operation)
n_de_rh_2w_all = prepare_network(n_de_rh_2w_all)
n_de_rh_2w_all.stores.e_cyclic = False
n_de_rh_2w_all = solve_network_de_rh_sp(n_de_rh_2w_all, n_operation, add_h2_bidding = True, add_battery_didding = True, rh_each_snapshot = False, rh_2w = True)

In [ ]:
where='2014'

fig, (ax1,ax2,ax3) = plt.subplots(3,1, figsize=(20,9))
plt.subplots_adjust(top = 0.9, bottom=0.1, hspace=.5, wspace=0)

n_operation.stores_t.e.loc[where].plot(ax=ax1)
ax1.set_title('operation, perfect foresight, no bidding prices')
ax1.set_xlabel('state of charge')

n_operation.buses_t.marginal_price.loc[where].plot(ax=ax2)
ax2.set_xlabel('marginal costs: buses')

n_de_rh_2w_h2.buses_t.marginal_price.loc[where].plot(ax=ax3)
ax3.set_xlabel('marginal costs: buses')

#fig.savefig(f'../plots/01-operation.jpeg')
plt.show()

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(3,1, figsize=(20,8))
plt.subplots_adjust(top = 0.9, bottom=0.1, hspace=.5, wspace=0)

n_de_rh.stores_t.e.plot(ax=ax1)
ax1.set_title('rolling horizon, 2 weeks foresight, no storage bidding prices')
ax1.set_xlabel('state of charge')

(n_operation.stores_t.e-n_de_rh.stores_t.e).plot(ax=ax2)
ax2.set_xlabel('state of charge: perfect foresight - rolling horizon')

n_de_rh_2w_h2.stores_t.e.plot(ax=ax3)
ax3.set_xlabel('state of charge: rolling horizon, 2 weeks foresight, bidding for H2')

#fig.savefig(f'../plots/02-rh.jpeg')
plt.show()

In [ ]:
where='2014'

fig, (ax1,ax2,ax3) = plt.subplots(3,1, figsize=(20,6))
plt.subplots_adjust(top = 0.9, bottom=0.1, hspace=.5, wspace=0)

n_de_rh_2w_h2.stores_t.e.loc[where].plot(ax=ax1)
ax1.set_title('rolling horizon, 2 weeks foresight, storage bidding for H2')
ax1.set_xlabel('state of charge')

n_de_rh_2w_h2.links_t.marginal_cost.loc[where].plot(ax=ax2)
ax2.set_xlabel('bidding prices')

(n_operation.stores_t.e-n_de_rh_2w_h2.stores_t.e).plot(ax=ax3)
ax3.set_xlabel('state of charge: perfect foresight - rolling horizon')

#fig.savefig(f'../plots/03-rh-h2.jpeg')
plt.show()

In [ ]:
where='2014'

fig, (ax1,ax2,ax3) = plt.subplots(3,1, figsize=(20,6))
plt.subplots_adjust(top = 0.9, bottom=0.1, hspace=.5, wspace=0)

n_de_rh_2w_all.stores_t.e.loc[where].plot(ax=ax1)
ax1.set_title('rolling horizon, 2 weeks foresight, storage bidding for H2+Battery')
ax1.set_xlabel('state of charge')

n_de_rh_2w_all.links_t.marginal_cost.loc[where].plot(ax=ax2)
ax2.set_xlabel('bidding prices')

(n_operation.stores_t.e-n_de_rh_2w_all.stores_t.e).plot(ax=ax3)
ax3.set_xlabel('state of charge: perfect foresight - snaphot-model')

#fig.savefig(f'../plots/{version}/04-rh-h2+b.jpeg')
plt.show()

In [ ]:
where='2014'

fig, (ax1,ax2,ax3) = plt.subplots(3,1, figsize=(20,6))
plt.subplots_adjust(top = 0.9, bottom=0.1, hspace=.5, wspace=0)

n_de_rh_2w_h2.stores_t.e.loc[where].plot(ax=ax1)
ax1.set_title('snapshot-model, no foresight, storage bidding for H2')
ax1.set_xlabel('state of charge')

n_de_rh_2w_h2.links_t.marginal_cost.loc[where].plot(ax=ax2)
ax2.set_xlabel('bidding prices')

(n_operation.stores_t.e-n_de_rh_2w_h2.stores_t.e).plot(ax=ax3)
ax3.set_xlabel('state of charge: perfect foresight - snaphot-model')

#fig.savefig(f'../plots/05-snapshot-h2.jpeg')
plt.show()

In [ ]:
# where='2014'

# fig, (ax1,ax2,ax3) = plt.subplots(3,1, figsize=(20,6))
# plt.subplots_adjust(top = 0.9, bottom=0.1, hspace=.5, wspace=0)

# n_de_rh_sp_all.stores_t.e.loc[where].plot(ax=ax1)
# ax1.set_title('snapshot-model, no foresight, storage bidding for H2+Battery')
# ax1.set_xlabel('state of charge')

# n_de_rh_sp_all.links_t.marginal_cost.loc[where].plot(ax=ax2)
# ax2.set_xlabel('bidding prices')

# (n_operation.stores_t.e-n_de_rh_sp_all.stores_t.e).plot(ax=ax3)
# ax3.set_xlabel('state of charge: perfect foresight - snaphot-model')

# fig.savefig(f'../plots/06-snapshot-h2+b.jpeg')
# plt.show()